In [53]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.style.use('seaborn-notebook')
%matplotlib inline

In [54]:
import src.scrubbing as scrubbing

In [55]:
df = pd.read_csv('data/Restaurant_Scores_-_LIVES_Standard.csv')

In [56]:
df2 = scrubbing.remove_missing_vid(df)

/home/soong/galv/capstone/SFeaters/src/scrubbing.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_viol['short_violation_id'] = L_vid


In [57]:
df3 = scrubbing.group_bid_idate(df2)

/home/soong/galv/capstone/SFeaters/src/scrubbing.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['inspect_date'] = pd.to_datetime(df.inspection_date)
/home/soong/galv/capstone/SFeaters/src/scrubbing.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['short_violation_id'] = df['short_violation_id'].apply(np.int64)


In [58]:
df_b = pd.read_csv('data/Registered_Business_Locations_-_San_Francisco.csv')
df4 = scrubbing.import_zipcode(df3, df_b)

In [59]:
df5 = scrubbing.get_zipcode_dummies(df4)

In [60]:
df6 = scrubbing.remove_rows_zero_violation(df5) 

## Test on scrub_all function

In [61]:
df7 = scrubbing.scrub_all(df)

/home/soong/galv/capstone/SFeaters/src/scrubbing.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_viol['short_violation_id'] = L_vid
/home/soong/galv/capstone/SFeaters/src/scrubbing.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['inspect_date'] = pd.to_datetime(df.inspection_date)
/home/soong/galv/capstone/SFeaters/src/scrubbing.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http:

### Test done

In [75]:
y = df7['y_label']
X = df7[['p10_12', 'p13_18', 'p19_24', 'p25_36', '94013', '94014', '94080', '94101', '94102', '94103', '94104',
       '94105', '94107', '94108', '94109', '94110', '94111', '94112', '94114',
       '94115', '94116', '94117', '94118', '94120', '94121', '94122', '94123',
       '94124', '94127', '94129', '94130', '94131', '94132', '94133', '94134',
       '94143', '94158']]

In [76]:
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size=0.25, random_state=38)

## Let's stash X_test and y_test away for only test set purpose. Split X_train and y_train again for train and validation.

In [77]:
X_train, X_validation, y_train, y_validation = train_test_split(X_tr, y_tr, test_size=0.25, random_state=28)

In [78]:
log_model = LogisticRegression()
log_model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [79]:
log_model.coef_

array([[-0.35559827,  0.01954942,  0.20886761,  0.08386524, -0.52650552,
        -0.48990932,  0.        ,  0.42310768,  0.02006984,  0.55577285,
         1.2869633 , -0.30644431, -0.04131389,  0.03161384,  0.62459175,
         1.52459705, -0.16720855,  0.46675473,  0.9403458 , -1.37798398,
         0.18306156, -0.53377899,  0.13927229,  0.        , -0.08925677,
         0.46055029,  1.28669253, -0.31189219,  0.4755737 ,  0.        ,
        -0.66980407,  0.19914285,  1.62859709,  0.46221792,  0.14321426,
         0.39764732, -0.94190598]])

In [80]:
log_model.intercept_

array([-0.2168155])

In [81]:
log_model.score(X_validation, y_validation)

0.6912599318955732

# Gradient Boosting Model

In [82]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    gb = GradientBoostingClassifier(n_estimators=20, learning_rate = learning_rate, max_features=2, max_depth = 2, random_state = 0)
    gb.fit(X_train, y_train)
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb.score(X_validation, y_validation)))

Learning rate:  0.05
Accuracy score (training): 0.606
Accuracy score (validation): 0.579
Learning rate:  0.1
Accuracy score (training): 0.637
Accuracy score (validation): 0.608
Learning rate:  0.25
Accuracy score (training): 0.679
Accuracy score (validation): 0.661
Learning rate:  0.5
Accuracy score (training): 0.691
Accuracy score (validation): 0.686
Learning rate:  0.75
Accuracy score (training): 0.699
Accuracy score (validation): 0.696
Learning rate:  1
Accuracy score (training): 0.702
Accuracy score (validation): 0.704


In [83]:
# Let's use learning rate of 0.75
gb = GradientBoostingClassifier(n_estimators=20, learning_rate = 0.75, max_features=2, max_depth = 2, random_state = 0)
gb.fit(X_train, y_train)
print("Accuracy score (training): {0:.3f}".format(gb.score(X_train, y_train)))
print("Accuracy score (validation): {0:.3f}".format(gb.score(X_validation, y_validation)))
# The "validation" here is a correct term. It is not a "test" set.

Accuracy score (training): 0.699
Accuracy score (validation): 0.696


## Find out if my data set is unbalanced or not

In [84]:
y.value_counts()

True     2791
False    1906
Name: y_label, dtype: int64

In [85]:
num_true = y.value_counts()[True]
num_false = y.value_counts()[False]

In [86]:
True_rate = num_true/(num_true + num_false)
True_rate

0.5942090696189057